<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import cross_val_score

In [2]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

In [3]:
train_df.head(10)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
5,c-8,c-3,c-4,1918,NW,MEM,MCO,683,N
6,c-1,c-27,c-4,754,DL,PBI,LGA,1035,N
7,c-4,c-29,c-6,635,OH,MSP,CVG,596,N
8,c-7,c-28,c-5,735,AA,ONT,DFW,1189,N
9,c-6,c-20,c-2,2029,OO,DEN,PSC,853,N


In [4]:
train_df['Distance'].max()

4962

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
Month                100000 non-null object
DayofMonth           100000 non-null object
DayOfWeek            100000 non-null object
DepTime              100000 non-null int64
UniqueCarrier        100000 non-null object
Origin               100000 non-null object
Dest                 100000 non-null object
Distance             100000 non-null int64
dep_delayed_15min    100000 non-null object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [6]:
c_days_mounth = [
    31,
    29,
    31,
    30,
    31,
    30,
    31,
    31,
    30,
    31,
    30,
    31
]

In [7]:
def season_grouping(mounth):
    mounth_index = int(mounth.split('-')[1])
    if (mounth_index) == 12 or (1 <= mounth_index <= 2):
        return 'winter'
    elif 3 <= mounth_index <= 5:
        return 'spring'
    elif 6 <= mounth_index <= 8:
        return 'summer'
    elif 9 <= mounth_index <= 11:
        return 'autumn'

    
def mounth_grouping(combo):
    mounth, day = combo.split('+')
    mounth_index = int(mounth.split('-')[1])
    day_index = int(day.split('-')[1])
    sub = day_index / c_days_mounth[mounth_index - 1]
    if sub <= (1 / 3):
        return 'start'
    elif sub <= (2 / 3):
        return 'middle'
    else:
        return 'end'

    
def weekend_grouping(day):
    day_index = int(day.split('-')[1])
    if day_index == 1 or day_index == 7: 
        return 'rest'
    else:
        return 'work'


def speed(ratio):
    if ratio < 0.5:
        return 'very long'
    elif ratio < 0.9:
        return 'long'
    elif ratio < 1.40:
        return 'mibble'
    elif ratio < 1.8:
        return 'fast'
    elif ratio >= 1.8:
        return 'very fast'
    
    
def speed2(ratio):
    if ratio < 0.3:
        return 'long'
    else:
        return 'fast'
    
def day_split(time):
    time = int(time)
    if time < 600:
        return 'c-1'
    elif time >= 600 and time < 1200:
        return 'c-2'
    elif time >= 1200 and time < 1800:
        return 'c-3'
    elif time >= 1800:
        return 'c-4'

def dep_grouping(time):
    time = int(time)
    if time < 100:
        return 'c-1'
    elif time >= 100 and time < 200:
        return 'c-2'
    elif time >= 200 and time < 300:
        return 'c-3'
    elif time >= 300 and time < 400:
        return 'c-4'
    elif time >= 400 and time < 500:
        return 'c-5'
    elif time >= 500 and time < 600:
        return 'c-6'
    elif time >= 600 and time < 700:
        return 'c-7'
    elif time >= 700 and time < 800:
        return 'c-8'
    elif time >= 800 and time < 900:
        return 'c-9'
    elif time >= 900 and time < 1000:
        return 'c-10'
    elif time >= 1000 and time < 1100:
        return 'c-11'
    elif time >= 1100 and time < 1200:
        return 'c-12'
    elif time >= 1200 and time < 1300:
        return 'c-13'
    elif time >= 1300 and time < 1400:
        return 'c-14'
    elif time >= 1400 and time < 1500:
        return 'c-15'
    elif time >= 1500 and time < 1600:
        return 'c-16'
    elif time >= 1600 and time < 1700:
        return 'c-17'
    elif time >= 1700 and time < 1800:
        return 'c-18'
    elif time >= 1800 and time < 1900:
        return 'c-19'
    elif time >= 1900 and time < 2000:
        return 'c-20'
    elif time >= 2000 and time < 2100:
        return 'c-21'
    elif time >= 2100 and time < 2200:
        return 'c-22'
    elif time >= 2200 and time < 2300:
        return 'c-23'
    elif time >= 2300:
        return 'c-24'
    
    
def dist_grouping(d):
    if d < 1000:
        return 'c-1'
    elif d >= 1000 and d < 2000:
        return 'c-2'
    elif d >= 2000 and d < 3000:
        return 'c-3'
    elif d >= 3000 and d < 4000:
        return 'c-4'
    elif d >= 4000:
        return 'c-5'

In [8]:
def time(time):
    time = str(time)
    if len(time) == 2:
        time = '00'+ time
    if len(time) == 3:
        time = '0'+ time
    if len(time) == 1:
        time = '0'+ time
    return time[:2] + ':' + time[2:]

tmp_train_df = pd.DataFrame(train_df[['Month', 'DayofMonth', 'DepTime']])
tmp_train_df['Time'] = tmp_train_df['DepTime'].apply(time)
tmp_train_df['Hours'] = tmp_train_df['Time'].apply(lambda x: x.split(':')[0])
tmp_train_df['Minutes'] = tmp_train_df['Time'].apply(lambda x: x.split(':')[1])
tmp_train_df['Year'] = 2018
tmp_train_df['Month'] = tmp_train_df['Month'].apply(lambda x: x.split('-')[1])
tmp_train_df['Day'] = tmp_train_df['DayofMonth'].apply(lambda x: x.split('-')[1])
tmp_train_df['Date'] = pd.to_datetime(tmp_train_df[['Month', 'Day', 'Year', 'Hours', 'Minutes']]) 
tmp_train_df = tmp_train_df.drop(['DayofMonth', 'DepTime'], axis=1)
tmp_train_df.head()

,Month,Time,Hours,Minutes,Year,Day,Date
0,8,19:34,19,34,2018,21,2018-08-21 19:34:00
1,4,15:48,15,48,2018,20,2018-04-20 15:48:00
2,9,14:22,14,22,2018,2,2018-09-02 14:22:00
3,11,10:15,10,15,2018,25,2018-11-25 10:15:00
4,10,18:28,18,28,2018,7,2018-10-07 18:28:00


In [9]:
tmp_test_df = pd.DataFrame(test_df[['Month', 'DayofMonth', 'DepTime']])
tmp_test_df['Time'] = tmp_test_df['DepTime'].apply(time)
tmp_test_df['Hours'] = tmp_test_df['Time'].apply(lambda x: x.split(':')[0])
tmp_test_df['Minutes'] = tmp_test_df['Time'].apply(lambda x: x.split(':')[1])
tmp_test_df['Year'] = 2018
tmp_test_df['Month'] = tmp_test_df['Month'].apply(lambda x: x.split('-')[1])
tmp_test_df['Day'] = tmp_test_df['DayofMonth'].apply(lambda x: x.split('-')[1])
tmp_test_df['Date'] = pd.to_datetime(tmp_test_df[['Month', 'Day', 'Year', 'Hours', 'Minutes']]) 
tmp_test_df = tmp_test_df.drop(['DayofMonth', 'DepTime'], axis=1)
tmp_test_df.head()

,Month,Time,Hours,Minutes,Year,Day,Date
0,7,06:15,06,15,2018,25,2018-07-25 06:15:00
1,4,07:39,07,39,2018,17,2018-04-17 07:39:00
2,12,06:51,06,51,2018,2,2018-12-02 06:51:00
3,3,16:14,16,14,2018,25,2018-03-25 16:14:00
4,6,15:05,15,05,2018,6,2018-06-06 15:05:00


In [10]:
#works
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

train_df['Date'] = tmp_train_df['Date']
train_df['Hours'] = train_df['Date'].apply(lambda x: x.hour)
train_df['Minutes'] = train_df['Date'].apply(lambda x: x.minute)
train_df = train_df.drop(['Date'], axis=1)
train_df['New'] = train_df['Origin'] + train_df['UniqueCarrier'] + train_df['Hours'].apply(lambda x: str(x))

test_df['Date'] = tmp_test_df['Date']
test_df['Hours'] = test_df['Date'].apply(lambda x: x.hour)
test_df['Minutes'] = test_df['Date'].apply(lambda x: x.minute)
test_df = test_df.drop(['Date'], axis=1)
test_df['New'] = test_df['Origin'] + test_df['UniqueCarrier'] + test_df['Hours'].apply(lambda x: str(x))

# train_df['Hours'] = train_df['DepTime'].apply(dep_grouping)
# test_df['Hours'] = test_df['DepTime'].apply(dep_grouping)

# test_df['New'] = test_df['Origin'] + test_df['UniqueCarrier'] + test_df['Hours']


# train_df['Is_smooth'] = train_df['Date'].apply(lambda x: x.minute)
# train_df['Is_smooth'] = train_df['Is_smooth'] + ' + ' + train_df['Origin']

# train_df['Origin'] = train_df['Origin'] + '+' + train_df['DepTime']
# test_df['Origin'] = test_df['Origin'] + '+' + test_df['DepTime']


# test_df = test_df.drop(['Hours'], axis=1)

# train_df['Distance'] = train_df['Distance'].apply(dist_grouping)
# test_df['Distance'] = test_df['Distance'].apply(dist_grouping)
# train_df['Distance'] = train_df['Distance'] + '+' +train_df['UniqueCarrier'] 
# test_df['Distance'] = test_df['Distance'] + '+' + test_df['UniqueCarrier']
# train_df['air_w'] = train_df['DepTime'] + '-->' + train_df['UniqueCarrier']
# test_df['air_w'] = test_df['DepTime'] + '-->' + test_df['UniqueCarrier']

# train_df['Year'] = 2018
# test_df['Year'] = 2018

# train_df['Month'] = train_df['Month'].apply(lambda x: int(x.split('-')[1]))
# test_df['Month'] = test_df['Month'].apply(lambda x: int(x.split('-')[1]))

# train_df['DayofMonth'] = train_df['DayofMonth'].apply(lambda x: int(x.split('-')[1]))
# test_df['DayofMonth'] = test_df['DayofMonth'].apply(lambda x: int(x.split('-')[1]))

# train_df['Day'] = train_df['DayofMonth'] 
# test_df['Day'] = test_df['DayofMonth'] 

# train_df['Date'] = pd.to_datetime(train_df.loc[:, ['Year', 'Month', 'Day']])
# test_df['Date'] = pd.to_datetime(train_df.loc[:, ['Year', 'Month', 'Day']])

# train_df['DayOfWeek'] = train_df['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
# test_df['DayOfWeek'] = test_df['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))


# train_df['air_week'] = train_df['DayOfWeek'] + '+' + train_df['UniqueCarrier']
# test_df['air_week'] = test_df['DayOfWeek'] + '+' + test_df['UniqueCarrier']
# train_df['air_month'] = train_df['Month'] + '+' + train_df['UniqueCarrier']
# test_df['air_month'] = test_df['Month'] + '+' + test_df['UniqueCarrier']

# train_df['speed'] = (train_df['Distance'] / train_df['DepTime']).apply(speed)

# train_df['DayofMonth'] = train_df['Month'] + '+' + train_df['DayofMonth']
# test_df['DayofMonth'] = test_df['Month'] + '+' + test_df['DayofMonth']
# train_df['DayofMonth'] = train_df['DayofMonth'].apply(mounth_grouping)
# test_df['DayofMonth'] = test_df['DayofMonth'].apply(mounth_grouping)

# train_df['Seasons'] = train_df['Month'].apply(season_grouping)
# test_df['Seasons'] = test_df['Month'].apply(season_grouping)

# train_df['Weekends'] = train_df['DayOfWeek'].apply(weekend_grouping)
# test_df['Weekends'] = test_df['DayOfWeek'].apply(weekend_grouping)

# train_df = train_df.drop(['DepTime', 'Distance'], axis=1)
# test_df = test_df.drop(['DepTime', 'Distance'], axis=1)

# train_df = train_df.drop(['Distance'], axis=1)
# test_df = test_df.drop(['Distance'], axis=1)

# train_df = train_df.drop(['DayofMonth'], axis=1)
# test_df = test_df.drop(['DayofMonth'], axis=1)
# train_df = train_df.drop(['DayOfWeek'], axis=1)
# test_df = test_df.drop(['DayOfWeek'], axis=1)

# train_df = train_df.drop(['Origin'], axis=1)
# test_df = test_df.drop(['Origin'], axis=1)

# train_df[''] = 
# test_df[''] = 
train_df[train_df['dep_delayed_15min'] == 'Y'].head(20)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,flight,Hours,Minutes,New
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW-->OMA,18.0,28.0,MDWWN18.0
18,c-10,c-14,c-5,1704,OO,SLC,SEA,689,Y,SLC-->SEA,17.0,4.0,SLCOO17.0
21,c-6,c-21,c-3,1519,EV,ATL,ILM,377,Y,ATL-->ILM,15.0,19.0,ATLEV15.0
33,c-12,c-3,c-7,1653,EV,ATL,MYR,317,Y,ATL-->MYR,16.0,53.0,ATLEV16.0
35,c-3,c-24,c-4,2303,US,CLT,CHS,168,Y,CLT-->CHS,23.0,3.0,CLTUS23.0
36,c-5,c-6,c-5,1728,WN,SAN,SMF,480,Y,SAN-->SMF,17.0,28.0,SANWN17.0
49,c-10,c-20,c-5,2312,AQ,OAK,LAS,407,Y,OAK-->LAS,23.0,12.0,OAKAQ23.0
50,c-9,c-9,c-6,1520,YV,SEA,PHX,1107,Y,SEA-->PHX,15.0,20.0,SEAYV15.0
58,c-9,c-5,c-2,1831,FL,ATL,MSP,906,Y,ATL-->MSP,18.0,31.0,ATLFL18.0
72,c-6,c-6,c-2,1932,UA,DEN,DTW,1123,Y,DEN-->DTW,19.0,32.0,DENUA19.0


In [11]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]

X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, 
    y_train, 
    test_size=0.3, 
    random_state=17
)


In [12]:
train_df.drop('dep_delayed_15min', axis=1).head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight,Hours,Minutes,New
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,ATL-->DFW,19.0,34.0,ATLAA19.0
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,PIT-->MCO,15.0,48.0,PITUS15.0
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,RDU-->CLE,14.0,22.0,RDUXE14.0
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,DEN-->MEM,10.0,15.0,DENOO10.0
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,MDW-->OMA,18.0,28.0,MDWWN18.0


In [13]:
train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object'

Month             True
DayofMonth        True
DayOfWeek         True
DepTime          False
UniqueCarrier     True
Origin            True
Dest              True
Distance         False
flight            True
Hours            False
Minutes          False
New               True
dtype: bool

In [14]:
categ_feat_idx

array([ 0,  1,  2,  4,  5,  6,  8, 11])

In [15]:
new_cat = CatBoostClassifier(
    random_state=17,
    silent=True
)
# learning_rate=0.05,
#     depth=10,
#     iterations=500,
#     l2_leaf_reg=9,

cvs = cross_val_score(
    new_cat, 
    X_train_part, 
    y_train_part, 
    scoring='roc_auc', 
    fit_params={'cat_features': categ_feat_idx},
    cv=5,
    n_jobs=-1
)
print('CV: ', cvs.mean())

new_cat.fit(X_train_part, y_train_part, cat_features=categ_feat_idx);
print('HO: ', roc_auc_score(y_valid, new_cat.predict_proba(X_valid)[:, 1]))

new_cat.fit(X_train, y_train, cat_features=categ_feat_idx);
new_cat_test_pred = new_cat.predict_proba(X_test)[:, 1]
# c разбиение на часы

CV:  0.8048532231976651
HO:  0.814196014697031


In [18]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv('sample_submission.csv', index_col='id')
    sample_sub['dep_delayed_15min'] = new_cat_test_pred
    sample_sub.to_csv('neW_ctb_pred.csv')

In [17]:
pd.DataFrame({'feature_name':train_df.drop('dep_delayed_15min', axis=1).columns, 'importance':new_cat.feature_importances_})

,feature_name,importance
0,Month,3.479454
1,DayofMonth,2.802458
2,DayOfWeek,2.746117
3,DepTime,12.118398
4,UniqueCarrier,5.812703
5,Origin,3.433951
6,Dest,9.990267
7,Distance,4.069521
8,flight,8.922673
9,Hours,8.129102
